In [ ]:
import openpyxl
import mysql.connector
from tkinter import *
from tkinter import messagebox, ttk

In [ ]:
def in_or_out(master, cursor, d_name):
    x = input('To export the database to a spreadsheet type 1, to manipulate data using spreadsheet type 2 - ')
    if (x == '1'):
        try:
            cursor.execute("SHOW TABLES;")
            tables = cursor.fetchall()
            wb = openpyxl.Workbook()
            wb.remove(wb[wb.sheetnames[0]])

            reference_metadata = []
            attr = []

            for i in tables:
                wb.create_sheet(title=i[0])
                cursor.execute("""SELECT 
                                  TABLE_NAME,COLUMN_NAME,CONSTRAINT_NAME, REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
                                FROM
                                  INFORMATION_SCHEMA.KEY_COLUMN_USAGE
                                WHERE
                                  REFERENCED_TABLE_SCHEMA = '"""+ d_name +"""' AND
                                  REFERENCED_TABLE_NAME = '""" + i[0] + "';")
                reference_metadata += cursor.fetchall()
                cursor.execute("SHOW columns FROM "+str(i[0])+";")
                metadata = cursor.fetchall()
                attributes = [column[0] for column in metadata]
                cursor.execute("SELECT * FROM "+str(i[0])+";")
                table = cursor.fetchall()
                attr.append(attributes)
                table = [tuple(attributes), [column[1] for column in metadata], [column[3] for column in metadata],[]] + table
                sheet = wb[i[0]]
                for j, j_item in enumerate(table):
                    for k, k_item in enumerate(j_item):
                        sheet.cell(row=j+1, column=k+1).value = k_item

            for i in reference_metadata:
                sheet = wb[i[0]]
                index = attr[tables.index((i[0],))].index(i[1]) + 1
                sheet.cell(row=4, column=index).value = "REFERENCES "+i[3]+'('+i[4]+')'
            y = input('Location for saving excel sheet with extension - ')
            wb.save(y)
        except:
            print ("There was an error, please try again.")
            

    elif (x=='2'):
        y = input('Location of the excel spreadsheet - ')
        cnx.autocommit = False

        try:
            cursor.execute("SET FOREIGN_KEY_CHECKS=0;")
            wb = openpyxl.load_workbook(y)
            cursor.execute("SHOW TABLES;")
            tables = [i[0].lower() for i in cursor.fetchall()]
            z = [j.lower() for j in wb.sheetnames]
            cursor.execute("""SELECT 
                                  TABLE_NAME,COLUMN_NAME,CONSTRAINT_NAME, REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
                                FROM
                                  INFORMATION_SCHEMA.KEY_COLUMN_USAGE
                                WHERE
                                  REFERENCED_TABLE_SCHEMA = '"""+ d_name+"';")
            metadata = cursor.fetchall()
            for k in metadata:
                cursor.execute("ALTER TABLE " + k[0] + " DROP FOREIGN KEY " + k[2] +";")
            for i in tables:
                if (i.lower() not in z):
                    cursor.execute("DROP TABLE "+i+";")
            cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
            sheets = wb.sheetnames
            for index, i in enumerate(sheets):
                primary = []
                prim_index = []
                foriegn = []
                sheet = wb[i]
                if (i.lower() not in tables):
                    query = "CREATE TABLE IF NOT EXISTS "+i+" ("
                    for j in range(sheet.max_column):
                        query += str(sheet.cell(row=1, column=j+1).value) + " " + str(sheet.cell(row=2, column=j+1).value)+","
                        if (str(sheet.cell(row=3, column=j+1).value) == 'PRI' or str(sheet.cell(row=3, column=j+1).value) == 'MUL'):
                            primary.append(sheet.cell(row=1, column=j+1).value)
                            prim_index.append(j+1)
                        if (str(sheet.cell(row=4, column=j+1).value)!= 'None'):
                            foriegn.append("FOREIGN KEY (" + str(sheet.cell(row=1, column=j+1).value)+ ") "+str(sheet.cell(row=4,column=j+1).value))
                    if (foriegn != []):
                        temp = ","+','.join(foriegn)
                    else:
                        temp = ""
                    query += "PRIMARY KEY ("+','.join(primary)+")"+temp+");"
                    cursor.execute(query)
                    
                else:
                    columns = []
                    types = []
                    for j in range(sheet.max_column):
                        columns.append(sheet.cell(row=1, column=j+1).value.lower())
                        types.append(sheet.cell(row=2, column=j+1).value)
                        if (str(sheet.cell(row=3, column=j+1).value) == 'PRI' or str(sheet.cell(row=3, column=j+1).value) == 'MUL'):
                            primary.append(sheet.cell(row=1, column=j+1).value.lower())
                            prim_index.append(j+1)
        
                    cursor.execute("SHOW columns FROM "+str(i)+";")
                    metadata = cursor.fetchall()
                    prim_database = [k[0] for k in metadata if (k[3]=='PRI' or k[3]=='MUL')]
                    if (primary != prim_database):
                        print (0/0)
                    head = [k[0] for k in metadata]
                    for k in metadata:
                        if (k[0].lower() not in columns):
                            cursor.execute("ALTER TABLE " + i +" DROP COLUMN "+k[0].lower()+";")
                    for ind,k in enumerate(columns):
                        if (k.lower() not in head):
                            cursor.execute("ALTER TABLE " + i +" ADD COLUMN "+ k.lower()+" "+ types[ind]+";")
                    cursor.execute("""SELECT 
                                          TABLE_NAME,COLUMN_NAME,CONSTRAINT_NAME, REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
                                        FROM
                                          INFORMATION_SCHEMA.KEY_COLUMN_USAGE
                                        WHERE
                                          REFERENCED_TABLE_SCHEMA = '"""+ d_name +"""' AND
                                          REFERENCED_TABLE_NAME = '""" + i + "';")
                    metadata = cursor.fetchall()
                    for k in metadata:
                        cursor.execute("ALTER TABLE " + k[0] + " DROP FOREIGN KEY " + k[2] +";")
                    
                            
                            
            sheets = wb.sheetnames
            for index, i in enumerate(sheets):
                #print (i)
                primary = []
                prim_index = []
                columns = []
                sheet = wb[i]
                for j in range(sheet.max_column):
                    columns.append(sheet.cell(row=1, column=j+1).value.lower())
                    if (str(sheet.cell(row=3, column=j+1).value) == 'PRI' or str(sheet.cell(row=3, column=j+1).value) == 'MUL'):
                        primary.append(sheet.cell(row=1, column=j+1).value.lower())
                        prim_index.append(j+1)
                pk_data = []
                data = []
                for j in range(5, sheet.max_row+1):
                    temp = []
                    temp1 = []
                    for k in range(1, sheet.max_column+1):
                        temp.append(sheet.cell(row=j, column=k).value)

                    for k in prim_index:
                        temp1.append(temp[k-1])
                    data.append(tuple(temp[:]))
                    pk_data.append(tuple(temp1[:]))
                    
                    #print (data)
                    #print (pk_data)
                
                cursor.execute("SELECT "+','.join(columns)+" FROM "+i.lower()+";")
                database_data = cursor.fetchall()

                cursor.execute("SHOW columns FROM "+str(i)+";")
                metadata = cursor.fetchall()
                #print (metadata)
                attributes = [zz for zz, column in enumerate(metadata) if (column[3]=='PRI' or column[3]=='MUL')]
                database_pk_data = []
                for yy in database_data:
                    temp = []
                    for zz in attributes:
                        temp.append(yy[zz])
                    database_pk_data.append(temp[:])
                #print (database_data)
                #print (database_pk_data)

                for index_j, j in enumerate(database_pk_data):
                    if (j not in pk_data):
                        cursor.execute(("DELETE FROM "+i.lower()+" WHERE ("+','.join(columns)+")<=>"+str(tuple(database_data[index_j]))+";").replace('None', 'NULL'))
                
                for index_j, j in enumerate(pk_data):
                    if (j not in database_pk_data):
                        #print ('step 1')
                        #print (("INSERT INTO "+i.lower()+" ("+','.join(columns)+") VALUES "+str(tuple(data[index_j]))+";").replace('None', 'NULL'))
                        cursor.execute(("INSERT INTO "+i.lower()+" ("+','.join(columns)+") VALUES "+str(tuple(data[index_j]))+";").replace('None', 'NULL'))
                        #print ('step 2')
                    else:
                        
                        if (data[index_j] not in database_data):
                            #print ('step 3')
                            cursor.execute(("DELETE FROM "+i.lower()+" WHERE ("+','.join(columns)+")<=>"+str(tuple(database_data[database_pk_data.index(j)]))+";").replace('None', 'NULL'))
                            cursor.execute(("INSERT INTO "+i.lower()+" ("+','.join(columns)+") VALUES "+str(tuple(data[index_j]))+";").replace('None', 'NULL'))
                            #print ('step 4')
                for j in range(wb[i].max_column):
                    if (str(sheet.cell(row=4, column=j+1).value) != 'None'):
                        cursor.execute("ALTER TABLE " + i + " ADD FOREIGN KEY (" + str(sheet.cell(row=1, column=j+1).value)+ ") "+ str(sheet.cell(row=4, column=j+1).value) +" ON DELETE CASCADE;")
            cnx.commit()

        except Exception as e:
            print (e)
            #print(("DELETE FROM "+i.lower()+" WHERE ("+','.join(columns)+")<=>"+str(tuple(database_data[database_pk_data.index(j)]))+";").replace('None', 'NULL'))
            #print(("INSERT INTO "+i.lower()+" ("+','.join(columns)+") VALUES "+str(tuple(data[index_j]))+";").replace('None', 'NULL'))
                            
            print ("Please check your excel sheet, you might have broken some constraint.")
            cnx.rollback()
            

    else:
        print ('Wrong input given, try again')

        

def create():
    global master, cursor
    master.destroy()
    x = input('Name of database - ')
    try:
        cursor.execute("CREATE DATABASE "+str(x)+";")
        cursor.execute("USE "+str(x)+";")
    except:
        print ('Either a database of same name exist or permission denied')
    in_or_out(master, cursor, x)

def modify():
    global master, cursor
    master.destroy()
    x = input('Name of database - ')
    try:
        cursor.execute("USE "+str(x)+";")
    except:
        print ('Either database does not exist or no permissions')
    in_or_out(master, cursor, x)


def delete():
    global master, cursor
    master.destroy()
    x = input('Name of database - ')
    y = input('To confirm type y and press enter - ')
    if (y == 'y'):
        try:
            cursor.execute("DROP DATABASE "+str(x)+";")
        except:
            print ('Either database does not exist or no permissions')
    else:
        return


def database_main():
    global master, cursor
    master = Tk()
    master.title("Select Operation")
    master.geometry('350x200')

    button = ttk.Button(master, text='Modify existing database', command=modify).grid(row=0, column=1, pady=10, padx=100)
    button = ttk.Button(master, text='Create new database', command=create).grid(row=1, column=1, pady=10)
    button = ttk.Button(master, text='Delete database', command=delete).grid(row=2, column=1, pady=10)
    mainloop( )
    return
    

def login():
    global cursor, cnx
    try:    
        cnx = mysql.connector.connect(user=str(e1.get()), password=str(e2.get()), host=str(e3.get()))
        cursor = cnx.cursor()
    except:
        messagebox.showerror('Error' ,'Wrong username or password')
        e1.delete(0, END)
        e2.delete(0, END)
        return
    messagebox.showinfo('Success','Successful login')
    master.destroy()
    database_main()
    return

In [ ]:
cnx = 0
master = Tk()
master.title("Login")
master.geometry('350x200')

Label(master, text="Username", padx=40, pady=10).grid(row=0)
Label(master, text="Password", padx=40).grid(row=1)
Label(master, text="Host address", pady=10).grid(row=2)

e1 = Entry(master)
e2 = Entry(master, show="*")
e3 = Entry(master)
e3.insert(END, 'localhost')

e1.grid(row=0, column=1, pady=20)
e1.focus()
e2.grid(row=1, column=1)
e3.grid(row=2, column=1, pady=20)

button = ttk.Button(master, text='Login', command=login).grid(row=3, column=1, pady=10)

mainloop( )
try:
    cnx.close()
except:
    pass